<a href="https://colab.research.google.com/github/Asmina-hub/-Image-Segmentation-with-Unet/blob/main/Image_Segmentation_with_U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from numpy import linalg as LA
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from tqdm import tqdm

In [3]:
def conv3x3(in_channels, out_channels, stride=1, padding=1, bias=True, groups=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size =3, stride=stride,padding=padding, bias=bias, groups=groups )

def upconv2x2(in_channels, out_channels, mode='transpose'):
  if mode == 'transpose':
    return nn.ConvTranspose2d(in_channels,out_channels,kernel_size=2, stride=2)
  else:
    return nn.Sequential( nn.Upsample(mode='bilinear',scale_factor=2), conv1x1(in_channels,out_channels))

def conv1x1(in_channels, out_channels, groups=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=1, groups=groups, stride=1)



In [5]:
class DownConv(nn.Module):
  def __init__(self, in_channel, out_channel, pooling=True):
    super(DownConv, self).__init__()
    self.in_channel=in_channel
    self.out_channel=out_channel
    self.pooling = pooling

    self.conv1 = conv3x3(self.in_channel, self.out_channel)
    self.conv2 = conv3x3(self.out_channel, self.out_channel)

  def forward(self, x):

    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    before_pooling = x
    if self.pooling:
      x = F.max_pool2d(x, kernel_size=2, stride=2)
    return x, before_pooling



In [12]:
class upConv(nn.Module):
  def __init__(self, in_channel, out_channel, up_mode='transpose'):
    super(upConv, self).__init__()
    self.in_channel = in_channel
    self.out_channel = out_channel
    self.up_mode = up_mode

    self.upconv = upconv2x2( self.in_channel, self.out_channel, mode= self.up_mode)
    self.conv1 = conv3x3(
            2*self.out_channel, self.out_channel)
    self.conv2 = conv3x3(self.out_channel, self.out_channel)



  def forward(self, from_down, from_up):
    up = self.upconv(from_up)
    x = torch.cat((up, from_down), 1)
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    return x






In [16]:
class UNet(nn.Module):
  """ Unet Architecture """
  def __init__(self, in_channel, class_no):
    super(UNet, self).__init__()
    self.down_convolution_1 = DownConv(in_channel, 64)
    self.down_convolution_2 = DownConv(64, 128)
    self.down_convolution_3 = DownConv(128, 256)
    self.down_convolution_4 = DownConv(256, 512)
    self.bottle_neck = DownConv(512, 1024, pooling=False)
    self.up_convolution_1 = upConv(1024, 512)
    self.up_convolution_2 = upConv(512, 256)
    self.up_convolution_3 = upConv(256, 128)
    self.up_convolution_4 = upConv(128, 64)
    self.out = conv1x1(64, class_no)

  def forward(self, x):
    x1, before_pooling1 = self.down_convolution_1(x)
    x2, before_pooling2 = self.down_convolution_2(x1)
    x3, before_pooling3 = self.down_convolution_3(x2)
    x4, before_pooling4 = self.down_convolution_4(x3)
    x5, _ = self.bottle_neck(x4)
    up1 = self.up_convolution_1(before_pooling4, x5)
    up2 = self.up_convolution_2(before_pooling3, up1)
    up3 = self.up_convolution_3(before_pooling2, up2)
    up4 = self.up_convolution_4(before_pooling1, up3)
    out = self.out(up4)
    return out

In [17]:
input_image = torch.rand((1,3,512,512))
model = UNet(3,10)
output = model(input_image)
print(output.size())

torch.Size([1, 10, 512, 512])
